In [1]:
import numpy as np
import random
import os
import argparse
from pathlib import Path
import scipy.stats
import itertools
from statistics import mean
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from main.layout import Layout
from main.algorithms import enum_layout_wo_rdt, init_S, coarse_to_fined
from main.auto_models import MTSeqBackbone, MTSeqModel
from data.nyuv2_dataloader_adashare import NYU_v2
from data.taskonomy_dataloader_adashare import Taskonomy

# prepare params

In [2]:
# backbone

# resnet34
# backbone_type = 'resnet'
# prototxt = 'models/deeplab_resnet34_adashare.prototxt'
# D = coarse_B = 5
# mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}

# mobilenet
backbone_type = 'mobilenet'
prototxt = 'models/mobilenetv2.prototxt'
D = coarse_B = 5
mapping = {0:[0,1,2,3,4,5,6], 1:[7,8,9,10,11,12,13,14,15,16,17], 2:[18,19,20,21,22], 
           3:[23,24,25,26,27,28,29,30], 4:[31], 5:[32]}

In [3]:
# data
# NYUv2
# data = 'NYUv2'
# dataset = NYU_v2('/mnt/nfs/work1/huiguan/lijunzhang/policymtl/data/NYUv2/', 'test', crop_h=321, crop_w=321)
# dataloader = DataLoader(dataset, 1, shuffle=False)
# tasks = ['segment_semantic', 'normal', 'depth_zbuffer']
# cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}
# T = len(tasks)
# layout_idx = 2 # ind. models

# Taskonomy
data = 'Taskonomy'
dataset = Taskonomy('/mnt/nfs/work1/huiguan/lijunzhang/policymtl/data/Taskonomy', 'test_small', crop_h=224, crop_w=224)
dataloader = DataLoader(dataset, 1, shuffle=False)
tasks = ['segment_semantic','normal','depth_zbuffer','keypoints2d','edge_texture']
cls_num = {'segment_semantic': 17, 'normal': 3, 'depth_zbuffer': 1, 'keypoints2d': 1, 'edge_texture': 1}
T = len(tasks)
layout_idx = 4 # ind. models

In [4]:
# ind. weights
ckpt_PATH = '/mnt/nfs/work1/huiguan/lijunzhang/multibranch/checkpoint/'
# weight_PATH = ckpt_PATH + 'NYUv2/baseline/NR/2/segment_semantic_normal_depth_zbuffer.model' # NYUv2 + Resnet34
# weight_PATH = ckpt_PATH + 'NYUv2/baseline/NM/2/segment_semantic_normal_depth_zbuffer.model' # NYUv2 + MobileNetV2
# weight_PATH = ckpt_PATH + 'Taskonomy/baseline/TR/4/segment_semantic_normal_depth_zbuffer_keypoints2d_edge_texture.model' # Taskonomy + Resnet34
weight_PATH = ckpt_PATH + 'Taskonomy/baseline/TM/4/segment_semantic_normal_depth_zbuffer_keypoints2d_edge_texture.model' # Taskonomy + MobileNetV2

In [5]:
# real results
real_date = '0216'

# procedure

In [6]:
with torch.no_grad():
    backbone = MTSeqBackbone(prototxt)
    fined_B = len(backbone.basic_blocks)
    feature_dim = backbone(torch.rand(1,3,224,224)).shape[1]

In [7]:
# layout
layout_list = [] 
S0 = init_S(T, coarse_B) # initial state
L = Layout(T, coarse_B, S0) # initial layout
layout_list.append(L)
enum_layout_wo_rdt(L, layout_list)

layout = layout_list[layout_idx]
print('Coarse Layout:', flush=True)
print(layout, flush=True)

layout = coarse_to_fined(layout, fined_B, mapping)
print('Fined Layout:', flush=True)
print(layout, flush=True)

# model
with torch.no_grad():
    model = MTSeqModel(prototxt, layout=layout, feature_dim=feature_dim, cls_num=cls_num)
    model = model.cuda()
    
# load ind. model weights
model.load_state_dict(torch.load(weight_PATH)['state_dict'])

Coarse Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]
Fined Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}

<All keys matched successfully>

In [8]:
# extract features
K = 500
D_pos = [mapping[d][-1] for d in range(D)]
D_feats = {task: [None]*D for task in tasks}

for i, batch in enumerate(dataloader):
    if i >= K:
        break
    
    with torch.no_grad():
        x = batch['input'].cuda()
        all_feats = model.extract_features(x)

    idx = 0
    for t in tasks:
        for d in range(D):
            vec_feat = all_feats[idx][D_pos[d]].cpu().numpy().reshape(1,-1)
            if D_feats[t][d] is None:
                D_feats[t][d] = vec_feat
            else:
                temp = D_feats[t][d]
                D_feats[t][d] = np.vstack((temp,vec_feat))
        idx += 1

In [9]:
# compute RDM
RDM = {task: np.zeros((D,K,K)) for task in tasks}
for t in tasks:
    for d in range(D):
        RDM[t][d,:,:] = 1 - np.corrcoef(D_feats[t][d])

In [10]:
# compute RSA
RSA = np.ones((D,T,T))
for d in range(D):
    for two_task in itertools.combinations(tasks, 2):
        rho = scipy.stats.spearmanr(np.tril(RDM[two_task[0]][d,:,:]).reshape(-1), np.tril(RDM[two_task[1]][d,:,:]).reshape(-1))[0]
        RSA[d,tasks.index(two_task[0]),tasks.index(two_task[1])] = RSA[d,tasks.index(two_task[1]),tasks.index(two_task[0])] = rho
print(RSA)

[[[1.         0.97331621 0.97712198 0.93423629 0.97425751]
  [0.97331621 1.         0.98040396 0.92660003 0.96293404]
  [0.97712198 0.98040396 1.         0.92793239 0.97474073]
  [0.93423629 0.92660003 0.92793239 1.         0.93417184]
  [0.97425751 0.96293404 0.97474073 0.93417184 1.        ]]

 [[1.         0.95615076 0.96493666 0.91267397 0.94029933]
  [0.95615076 1.         0.98065669 0.89710142 0.91682761]
  [0.96493666 0.98065669 1.         0.90022651 0.92271932]
  [0.91267397 0.89710142 0.90022651 1.         0.90509235]
  [0.94029933 0.91682761 0.92271932 0.90509235 1.        ]]

 [[1.         0.94599522 0.94899431 0.91582797 0.93470601]
  [0.94599522 1.         0.98084947 0.89741627 0.90988288]
  [0.94899431 0.98084947 1.         0.89773325 0.9070251 ]
  [0.91582797 0.89741627 0.89773325 1.         0.91776544]
  [0.93470601 0.90988288 0.9070251  0.91776544 1.        ]]

 [[1.         0.92282857 0.92715091 0.87015332 0.88078254]
  [0.92282857 1.         0.96287514 0.86820405 0.8

In [11]:
# set score for each layout
for layout in layout_list:
    score = 0
    for d in range(D):
        C_cluster = []
        for task_set in layout[d]:
            max_dis = 0
            for two_task in itertools.combinations(task_set, 2):
                max_dis = max(max_dis, 1-RSA[d,two_task[0],two_task[1]])
            C_cluster.append(max_dis)
        score += mean(C_cluster)
    layout.score = score

In [12]:
# sort layout
layout_order = sorted(range(len(layout_list)), key=lambda k: layout_list[k],reverse=False)
step = 1
for i in range(0,len(layout_order),step):
    print(layout_order[i])
    L = layout_list[layout_order[i]]
    print(L)
    print(L.score)
    print('=' * 100)

4
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]
0
150
[[{0}, {1, 2}, {4}, {3}], [{0}, {4}, {3}, {2}, {1}], [{0}, {4}, {3}, {2}, {1}], [{0}, {4}, {3}, {2}, {1}], [{0}, {4}, {3}, {2}, {1}]]
0.004899009617744671
1285
[[{0, 2}, {1}, {4}, {3}], [{1}, {4}, {3}, {2}, {0}], [{1}, {4}, {3}, {2}, {0}], [{1}, {4}, {3}, {2}, {0}], [{1}, {4}, {3}, {2}, {0}]]
0.005719506227688076
10
[[{0}, {1}, {3}, {2, 4}], [{0}, {1}, {3}, {4}, {2}], [{0}, {1}, {3}, {4}, {2}], [{0}, {1}, {3}, {4}, {2}], [{0}, {1}, {3}, {4}, {2}]]
0.0063148180371243035
565
[[{0, 4}, {1}, {3}, {2}], [{1}, {3}, {2}, {4}, {0}], [{1}, {3}, {2}, {4}, {0}], [{1}, {3}, {2}, {4}, {0}], [{1}, {3}, {2}, {4}, {0}]]
0.0064356236189662686
2275
[[{0, 1}, {2}, {4}, {3}], [{2}, {4}, {3}, {1}, {0}], [{2}, {4}, {3}, {1}, {0}], [{2}, {4}, {3}, {1}, {0}], [{2}, {4}, {3}, {1}, {0}]]
0.006670948091594298
1540
[[{0, 2, 4}, {3}, {1}], [{3}, {1}, {0}, {4}, {2}], [{3}, 

In [13]:
# SROCC
def load_obj(name):
    with open('./ntask/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

real_log = load_obj('real_results_'+data+'_'+backbone_type+'_' + real_date)
idx = real_log['layout']
real_results = real_log['val_acc']
real = np.mean(real_results[:], axis=1)

est = [-layout_list[i].score for i in idx]
scipy.stats.spearmanr(est,real)

SpearmanrResult(correlation=-0.7545454545454545, pvalue=0.00728204100842914)

# Qizheng

In [5]:
weights = torch.load(weight_PATH)['state_dict']

In [27]:
D = 0
for key in weights:
    if 'weight' in key and 'backbone' in key:
        D += 1
D = int(D/3)

D_params = {task: [] for task in tasks}
for key in weights:
    if 'weight' in key and 'backbone' in key:
        t_idx = int(key.split('.')[2]) % 3
        D_params[tasks[t_idx]].append(weights[key].cpu().numpy().reshape(-1))

assert len(D_params[tasks[0]]) == len(D_params[tasks[1]]) == len(D_params[tasks[2]]) == D

In [28]:
# compute PSA
PSA = np.ones((D,T,T))
for d in range(D):
    for two_task in itertools.combinations(tasks, 2):
#         rho = scipy.stats.spearmanr(D_params[two_task[0]][d], D_params[two_task[1]][d])[0]
        rho = np.linalg.norm(D_params[two_task[0]][d] - D_params[two_task[1]][d])
        PSA[d,tasks.index(two_task[0]),tasks.index(two_task[1])] = PSA[d,tasks.index(two_task[1]),tasks.index(two_task[0])] = rho
print(PSA)

[[[ 1.          5.02830696  4.037117  ]
  [ 5.02830696  1.          3.63590288]
  [ 4.037117    3.63590288  1.        ]]

 [[ 1.          2.27176285  6.36746168]
  [ 2.27176285  1.          5.70963478]
  [ 6.36746168  5.70963478  1.        ]]

 [[ 1.          8.24909496  6.26709414]
  [ 8.24909496  1.          5.65949821]
  [ 6.26709414  5.65949821  1.        ]]

 [[ 1.          1.55947053  5.54519272]
  [ 1.55947053  1.          5.11428213]
  [ 5.54519272  5.11428213  1.        ]]

 [[ 1.          7.70594931  5.81748295]
  [ 7.70594931  1.          5.33898735]
  [ 5.81748295  5.33898735  1.        ]]

 [[ 1.          1.7155509   5.9160285 ]
  [ 1.7155509   1.          4.93741941]
  [ 5.9160285   4.93741941  1.        ]]

 [[ 1.          5.39825487  4.28039837]
  [ 5.39825487  1.          3.43453503]
  [ 4.28039837  3.43453503  1.        ]]

 [[ 1.          2.56993985  4.3446207 ]
  [ 2.56993985  1.          3.0612309 ]
  [ 4.3446207   3.0612309   1.        ]]

 [[ 1.          5.346772

In [54]:
# 'segment_semantic', 'normal', 'depth_zbuffer'
distance_sum = []
for d in range(D):
    distance_sum.append(PSA[d,0,2])
print('segment_semantic vs normal')
print(mean(distance_sum))
print(max(distance_sum))
print(min(distance_sum))
print('=' * 20)

distance_sum = []
for d in range(D):
    distance_sum.append(PSA[d,0,1])
print('segment_semantic vs depth_zbuffer')
print(mean(distance_sum))
print(max(distance_sum))
print(min(distance_sum))
print('=' * 20)

distance_sum = []
for d in range(D):
    distance_sum.append(PSA[d,1,2])
print('normal vs depth_zbuffer')
print(mean(distance_sum))
print(max(distance_sum))
print(min(distance_sum))
print('=' * 20)

segment_semantic vs normal
5.4316812422540455
20.918720245361328
0.6082245707511902
segment_semantic vs depth_zbuffer
5.506681526700656
22.134845733642578
0.6820645928382874
normal vs depth_zbuffer
3.010250359773636
8.058507919311523
0.27812889218330383


In [30]:
np.min(PSA)

0.27812889218330383

In [29]:
# PSA[PSA==1] = 0
np.max(PSA)

22.134845733642578